In [2]:
!pip install langchain
!pip install langchain-community
!pip install langchain-openai
!pip install langchainhub

In [3]:
# uncomment this if running locally
# from dotenv import load_dotenv

# load_dotenv()

# Or if you are in Colab, uncoment below and add your api key
import os
os.environ["OPENAI_API_KEY"] = "your-api-key"

The more complex version of this would use this pygithub package as shown in here:
- https://python.langchain.com/docs/integrations/toolkits/github

But let's try a simpler version that just sets up a bunch of python functions to automate different github actions like add, commit and pull requests.

Let's start with a simple commit to a branch of some repository.

First, I'll create a github repository using some simple commands.

In [4]:
# git init -b main

# git add . && git commit -m "Some commit"

# gh repo create

Now, let's try asking an agent to write come code and commit the resulting code to the current branch of this repository.

To do that, let's give the agent the necessary tools it will need which in this case will be python functions that perform different github actions using the `subprocess` package.

We'll follow the basic steps for building a langchain agent:

# Steps for building a simple agent:
- Set up the LLM
- Define the tool or toolkit (list of tools)
- Set up a prompt template
- Connect llm with your tools
- Define your agent as a dict with keys: input, and agent scratchpad 
- Use the Langchain LCEL language to pipe agent, prompt the llm_with_tools variable and the output parser (which can use OpenAI function calling parser)
- Create the agent loop
- Wrap everything into the AgentExecutor
- Invoke the agent with some query input


In [5]:
# setup the llm
from langchain_openai.chat_models import ChatOpenAI

llm_chat = ChatOpenAI(temperature=0)

Now, let's create a tool for the LLM

In [6]:
# !pip install PyGithub
# I don't want more external packages

In [7]:
import subprocess

def github_commit_tool(commit_message="Some commit"):
    subprocess.run(["git", "add", "."])
    subprocess.run(["git", "commit", "-m", commit_message])
    subprocess.run(["git", "push", "-u", "origin", "main"])
    
    return "Committed to Github"

Now! Before we use it with langchain, let's test it by itself.

In [8]:
# github_commit_tool("Some testing commit")

Now, let's take a look at out github repository from the terminal.

![](2023-10-29-20-47-54.png)

Nice! It looks like we are good to go with this first simple tool!

Let's now make it a tool for our langchain agent by adding the @tool. 

In [9]:
from langchain.tools import tool

@tool
def github_commit_tool(commit_message="Some commit"):
    """This function uses the subprocess package to make commits to a github repo pre-defined."""
    subprocess.run(["git", "add", "."])
    subprocess.run(["git", "commit", "-m", commit_message])
    subprocess.run(["git", "push", "-u", "origin", "main"])
    
    return "Committed to Github"

tools = [github_commit_tool]

We added some documentation to our function to abide by the requirements of the tool decorator from langchain.

Now, let's test if a simple agent can use that tool!

Let's start by setting up our prompt template.

In [10]:
# from langchain.agents import AgentExecutor
# from langchain.prompts import ChatPromptTemplate

# template = """
# You are a helpful programming assistant. The user will ask you to perform commits to a github repo and you will perform the needed actions and call the necessary
# functions when needed.
# Your current task:
# {input}
# """
# prompt = ChatPromptTemplate.from_template(template)

In [11]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [12]:
from langchain.agents import create_tool_calling_agent
from langchain_openai import ChatOpenAI

llm_chat = ChatOpenAI(temperature=0)
agent = create_tool_calling_agent(llm_chat, tools, prompt)

In [13]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent,tools=tools, verbose=True)

In [14]:
import subprocess

agent_executor.invoke({"input": "Create a github commit with the message 'Pancakes are better than waffles! And I hope I am not a super boring instructor!"})



> Entering new AgentExecutor chain...

Invoking: `github_commit_tool` with `{'commit_message': 'Pancakes are better than waffles! And I hope I am not a super boring instructor!'}`


[main ea965d3] Pancakes are better than waffles! And I hope I am not a super boring instructor!
 28 files changed, 3413 insertions(+), 965 deletions(-)
 create mode 100644 notebooks/7.0-langgraph-walkthrough.ipynb
 create mode 100644 notebooks/agent-deploy/.gitignore
 create mode 100644 notebooks/agent-deploy/Dockerfile
 create mode 100644 notebooks/agent-deploy/README.md
 create mode 100644 notebooks/agent-deploy/app/__init__.py
 create mode 100644 notebooks/agent-deploy/app/server.py
 create mode 100644 notebooks/agent-deploy/packages/README.md
 create mode 100644 notebooks/agent-deploy/packages/openai-functions-agent/LICENSE
 create mode 100644 notebooks/agent-deploy/packages/openai-functions-agent/README.md
 create mode 100644 notebooks/agent-deploy/packages/openai-functions-agent/main.py
 create mode

remote: 
remote: GitHub found 11 vulnerabilities on EnkrateiaLucca/oreilly_live_training_agents's default branch (1 high, 8 moderate, 2 low). To find out more, visit:        
remote:      https://github.com/EnkrateiaLucca/oreilly_live_training_agents/security/dependabot        
remote: 
To https://github.com/EnkrateiaLucca/oreilly_live_training_agents.git
   5e01c8c..ea965d3  main -> main


branch 'main' set up to track 'origin/main'.
Committed to GithubI have successfully created a GitHub commit with the message 'Pancakes are better than waffles! And I hope I am not a super boring instructor!'.

> Finished chain.


{'input': "Create a github commit with the message 'Pancakes are better than waffles! And I hope I am not a super boring instructor!",
 'output': "I have successfully created a GitHub commit with the message 'Pancakes are better than waffles! And I hope I am not a super boring instructor!'."}

And voila, the agent calls the right function and creates the commit!

(ignore the github related error, let's focus on the agent capabilities).

And voila! We made a commit!

![](2023-10-30-13-04-20.png)

Ok, this is cool but, we can go beyond that, and add the ability to create a file and then commit that file to our desired repository. Let's do that!

To do it all we have to do is amplify the Agent's toolkit with another tool that can create files in the current folder.

Perfect! Now, let's add this to our github agent

In [15]:
@tool
def create_file_tool(file_contents):
    """This function creates a file with the contents and file path provided as inputs."""
    with open("file.py", "w") as f:
        f.write(file_contents)
    
    return "File created"

Now, let's add that functionality to our agent by just updating the tools list.

In [16]:
tools = [github_commit_tool, create_file_tool]

Now, let's modify our agent with its new toolkit

In [17]:
agent = create_tool_calling_agent(llm_chat, tools, prompt)

In [18]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

Now, let's try using both tools in a call to see if the agent can do something slightly more complex now.

In [19]:
agent_executor.invoke({
    "input": "Write Python code with a for loop stating the truth about what is better: pancakes or waffles.\
    Output ONLY THE CODE and save that to a file then commit it to our github repository."
})



> Entering new AgentExecutor chain...

Invoking: `create_file_tool` with `{'file_contents': "# Python code\n\ntruths = ['Pancakes are better than waffles.', 'Waffles are better than pancakes.']\n\nfor truth in truths:\n    print(truth)\n", 'file_path': 'pancakes_vs_waffles.py'}`


File created
Invoking: `github_commit_tool` with `{'commit_message': 'Add Python code to state the truth about pancakes vs waffles'}`


On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   ../presentation-slides/presentation-llm-agents.html

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	../Makefile 2

no changes added to commit (use "git add" and/or "git commit -a")


Everything up-to-date


branch 'main' set up to track 'origin/main'.
Committed to GithubThe Python code has been written, saved to a file, and committed to your GitHub repository.

> Finished chain.


{'input': 'Write Python code with a for loop stating the truth about what is better: pancakes or waffles.    Output ONLY THE CODE and save that to a file then commit it to our github repository.',
 'output': 'The Python code has been written, saved to a file, and committed to your GitHub repository.'}

And voila again!
![](2023-10-30-13-15-38.png)

For The Practice Demo We'll stop here, below I show how to expand this for multi-input tools.

Ok! This is awesome right? Let's just stop for a second and appreciate the simplicity of this implementation and the power that it gives us! Now, on the other hand, we are using a [ZeroShotReactAgent](https://python.langchain.com/docs/modules/agents/agent_types/react#:~:text=Using-,zeroshotreactagent), which is limited when it comes to multi input functions as tools. However, we can easily expand this by leveraging Langchain's amazing [Multi-Input Tools](https://python.langchain.com/docs/modules/agents/tools/multi_input_tool#:~:text=On%20this%20page-,Multi-Input%20Tools,with%20the%20StructuredTool%20class.,-import%20os).

Let's do that to modify our current tool: `create_file_tool()` to accept the file's contents as well as a file path, this way we are not restricted in creating just one type of file with a pre-specified name.

In [20]:
from langchain.tools import StructuredTool

def create_file_tool(file_contents: str, file_path: str) -> str:
    """
    Function that creates a file with the contents 
    and file path provided as inputs.
    """
    with open(file_path, "w") as f:
        f.write(file_contents)
    
    return "File created"

# Let's test it first!
create_file_tool("print('Hello World')", "print_file.py")

'File created'

In [21]:
!ls *.py

file.py                   jira-agent.py             research-assistant.py
function_calls_chatgpt.py print_file.py


In [22]:
!python print_file.py

Hello World


Great! Now, let's make it a tool for the agent.

In [23]:
create_file_tool = StructuredTool.from_function(create_file_tool)

And, again, let's update our toolkit and our agent.

In [24]:
tools = [github_commit_tool, create_file_tool]

In [25]:
agent = create_tool_calling_agent(llm_chat, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

Now, let's test it by asking it to write a simple `README.md` file, giving the agent some context about the project's repo.

In [26]:
agent_executor.invoke({
    "input": "Write a report on why pancakes are so much better than all the other options (including sarcastially waffles) for a good happy breakfast. Save that to pancakes_for_the_win.md and commit it to our github repository."
})



> Entering new AgentExecutor chain...

Invoking: `create_file_tool` with `{'file_contents': "# Pancakes for the Win!\n\nPancakes are undoubtedly the superior choice for a happy breakfast. Here are a few reasons why pancakes reign supreme:\n\n1. Versatility: Pancakes can be customized with various toppings such as fruits, chocolate chips, syrup, and more. The possibilities are endless!\n\n2. Fluffiness: Nothing beats the fluffy texture of a well-made pancake. It's like biting into a cloud of happiness.\n\n3. Comfort: Pancakes evoke feelings of warmth and comfort, making them the perfect choice for a cozy breakfast.\n\n4. Nostalgia: Pancakes are a classic breakfast option that brings back fond memories of childhood and family gatherings.\n\nIn conclusion, pancakes are the ultimate breakfast choice that surpasses all other options, even sarcastically waffles. #PancakesForTheWin", 'file_path': 'pancakes_for_the_win.md'}`


File created
Invoking: `github_commit_tool` with `{'commit_messag

remote: 
remote: GitHub found 20 vulnerabilities on EnkrateiaLucca/oreilly_live_training_agents's default branch (2 high, 14 moderate, 4 low). To find out more, visit:        
remote:      https://github.com/EnkrateiaLucca/oreilly_live_training_agents/security/dependabot        
remote: 
To https://github.com/EnkrateiaLucca/oreilly_live_training_agents.git
   ea965d3..4fd5fb2  main -> main


branch 'main' set up to track 'origin/main'.
Committed to GithubThe report on why pancakes are the best for breakfast has been written, saved as "pancakes_for_the_win.md", and committed to our GitHub repository.

> Finished chain.


{'input': 'Write a report on why pancakes are so much better than all the other options (including sarcastially waffles) for a good happy breakfast. Save that to pancakes_for_the_win.md and commit it to our github repository.',
 'output': 'The report on why pancakes are the best for breakfast has been written, saved as "pancakes_for_the_win.md", and committed to our GitHub repository.'}

And again we succeed!

![](2023-10-30-13-26-32.png)

Ok, before we finish with this example, let's add a last tool to read files from a repo, for situations where we might write some Python code and we want our agent to integrate that Python code into our repository. 

Again, we follow the same procedure:

- Create the tool
- Update our tookit
- Update our agent
- Run a test task to check if it works 

In [27]:
def read_file_tool(file_path):
    with open(file_path, "r") as f:
        contents = f.read()
    
    return contents

read_file_tool("./pancakes_for_the_win.md")

"# Pancakes for the Win!\n\nPancakes are undoubtedly the superior choice for a happy breakfast. Here are a few reasons why pancakes reign supreme:\n\n1. Versatility: Pancakes can be customized with various toppings such as fruits, chocolate chips, syrup, and more. The possibilities are endless!\n\n2. Fluffiness: Nothing beats the fluffy texture of a well-made pancake. It's like biting into a cloud of happiness.\n\n3. Comfort: Pancakes evoke feelings of warmth and comfort, making them the perfect choice for a cozy breakfast.\n\n4. Nostalgia: Pancakes are a classic breakfast option that brings back fond memories of childhood and family gatherings.\n\nIn conclusion, pancakes are the ultimate breakfast choice that surpasses all other options, even sarcastically waffles. #PancakesForTheWin"

Ok, our function seems to work so let's add it to our toolkit, since its a simple function with only one input we can just add the @tool decorator on top of it.

In [28]:
@tool
def read_file_tool(file_path):
    """
    Simple function to read contents from a file.
    """
    with open(file_path, "r") as f:
        contents = f.read()
    
    return contents

And we update our toolkit:

In [29]:
tools = [github_commit_tool, create_file_tool, read_file_tool]

And we update our agent

In [30]:
agent = create_tool_calling_agent(llm_chat, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

Before we run our newly updated agent, let's create a .txt file containing the contents from this notebook so that our agent has something useful to read.

In [31]:
agent_executor.invoke(
    {
    "input": "Write a simple essay that counters all the arguments in the\
        pancakes_for_the_win.md file arguing for waffles, write that to waffles_for_the_win.md and commit it to our github repository."
    }
                      )



> Entering new AgentExecutor chain...

Invoking: `read_file_tool` with `{'file_path': 'pancakes_for_the_win.md'}`


# Pancakes for the Win!

Pancakes are undoubtedly the superior choice for a happy breakfast. Here are a few reasons why pancakes reign supreme:

1. Versatility: Pancakes can be customized with various toppings such as fruits, chocolate chips, syrup, and more. The possibilities are endless!

2. Fluffiness: Nothing beats the fluffy texture of a well-made pancake. It's like biting into a cloud of happiness.

3. Comfort: Pancakes evoke feelings of warmth and comfort, making them the perfect choice for a cozy breakfast.

4. Nostalgia: Pancakes are a classic breakfast option that brings back fond memories of childhood and family gatherings.

In conclusion, pancakes are the ultimate breakfast choice that surpasses all other options, even sarcastically waffles. #PancakesForTheWin
Invoking: `create_file_tool` with `{'file_contents': "# Waffles for the Win!\n\nWaffles may have th

remote: 
remote: GitHub found 20 vulnerabilities on EnkrateiaLucca/oreilly_live_training_agents's default branch (2 high, 14 moderate, 4 low). To find out more, visit:        
remote:      https://github.com/EnkrateiaLucca/oreilly_live_training_agents/security/dependabot        
remote: 
To https://github.com/EnkrateiaLucca/oreilly_live_training_agents.git
   4fd5fb2..4398abb  main -> main


branch 'main' set up to track 'origin/main'.
Committed to GithubI have written a simple essay countering the arguments in favor of waffles and highlighted the superiority of pancakes. The essay has been saved in a file named "waffles_for_the_win.md" and committed to our GitHub repository.

> Finished chain.


{'input': 'Write a simple essay that counters all the arguments in the        pancakes_for_the_win.md file arguing for waffles, write that to waffles_for_the_win.md and commit it to our github repository.',
 'output': 'I have written a simple essay countering the arguments in favor of waffles and highlighted the superiority of pancakes. The essay has been saved in a file named "waffles_for_the_win.md" and committed to our GitHub repository.'}

I think this is a great usecase for agents like these even if sometimes the performance and reliability is a bit off, because now, its a matter of optimizing the edges to get it to work for complex use cases.